In [1]:
import numpy as np

size=8000
x=np.random.normal(size=size).astype(np.float32)

%timeit np.sort(x,kind='heapsort')

680 µs ± 8.72 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [2]:
import jax 
import jax.numpy as jnp

key = jax.random.PRNGKey(0)
x = jax.random.normal(key, (size,))
%timeit jnp.sort(x,kind='heapsort').block_until_ready()

/usr/local/lib/python3.10/dist-packages/jax/_src/numpy/lax_numpy.py:3570: UserWarning: 'kind' argument to sort is ignored.
  warnings.warn("'kind' argument to sort is ignored.")


295 µs ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
x.device()

StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)

In [4]:
class abstract_graph:
    
    def __init__(self,_edges):
        self.edges=_edges
        self.nodes={u for u,v in self.edges} | {v for u,v in self.edges}
        
    def adjacency_matrix(self):
        pass
    
    def adjacency_list(self):
        pass

    
class simple_graph(abstract_graph):
    
    def __init__(self,_edges):
        tmp=[]
        for (u,v) in _edges:
            tmp.append((u,v))
            if (v,u) not in tmp and v!=u:
                tmp.append((v,u))
        self.edges=tmp
        self.nodes={u for u,v in _edges} | {v for u,v in _edges}
     
    def adjacency_matrix(self):
        # completar
        n=len(self.nodes)
        mat=np.zeros((n,n))
        for i,v in enumerate(self.nodes):
            for j,k in enumerate(self.nodes):
                if (v,k) in self.edges:
                    mat[i,j]=1
        return mat
    
    
    def adjacency_list(self):
        adjacent=lambda n : {v for u,v in self.edges if u==n } 
        return {v:adjacent(v) for v in self.nodes}

  
class weighted_graph(simple_graph):
    
    def __init__(self,_edges):
        tmp=dict()
        for (u,v),w in _edges.items():
            tmp.update({(u,v):w})
            if (v,u) not in tmp.keys() and v!=u:
                tmp.update({(v,u):w})
        self.edges=tmp
        self.nodes={u for u,v in _edges} | {v for u,v in _edges}

In [5]:
def find(C, u): 
    if C[u] != u:
        C[u] = find(C, C[u])
    return C[u]

def union(C, R, u, v):
    u, v = find(C, u), find(C, v)
    if R[u] > R[v]:
        C[v] = u 
    else:
        C[u] = v
    if R[u] == R[v]:
        R[v] += 1
        
def jax_kruskal(G):
    E = G.edges
    T=[]
    values=jnp.array([*E.values()])
    items=[*E.keys()]
    C,R = {u:u for u in G.nodes}, {u:0 for u in G.nodes}
    sorted_edges=[(items[c.item()],w.item()) for c,w in zip(jnp.argsort(values),jnp.sort(values))] 
    for (u,v),weight in sorted_edges:
        if find(C, u) != find(C, v): 
            T.append((u,v,{'weight':weight}))
            union(C, R, u, v)
    return T

# Algoritmo Kruskal en JAX

In [6]:
E={('a','b'):4,('b','c'):7,('b','d'):8,('c','d'):10,('a','c'):9}
G=weighted_graph(E)
print('aristas : ',G.edges)

aristas :  {('a', 'b'): 4, ('b', 'a'): 4, ('b', 'c'): 7, ('c', 'b'): 7, ('b', 'd'): 8, ('d', 'b'): 8, ('c', 'd'): 10, ('d', 'c'): 10, ('a', 'c'): 9, ('c', 'a'): 9}


In [7]:
T=jax_kruskal(G)

In [8]:
T

[('a', 'b', {'weight': 4}),
 ('b', 'c', {'weight': 7}),
 ('b', 'd', {'weight': 8})]

In [9]:
import networkx as nx

G_nx=nx.Graph()
G_nx.add_weighted_edges_from([(u,v,w) for ((u,v),w) in E.items()])
T_nx=nx.minimum_spanning_tree(G_nx,algorithm='kruskal')

In [10]:
T_nx.edges()

EdgeView([('a', 'b'), ('b', 'c'), ('b', 'd')])

# Algoritmo Prim en JAX

In [34]:
def jax_prim(graph,start):
    nodes=list(G.nodes)
    parents={n:None for n in nodes}
    tree=[]
    frontier=jnp.array([jnp.inf]*len(nodes))
    pos=nodes.index(start)
    frontier=frontier.at[pos].set(0.0)
    adjacency=graph.adjacency_list()
    visited=set()
    for _ in range(len(nodes)):
        pos=jnp.argmin(frontier[jnp.where(frontier >= 0)]).item()
        print(pos)
        weight=frontier.at[pos].get().item()
        v=nodes[pos]
        if v in visited:
            continue
        visited.update({v})
        u=parents[v]
        tree.append((u,v,{'weight':weight}))
        for neighbor in adjacency[v]:
            if neighbor not in visited:
                n_pos=nodes.index(neighbor)
                frontier=frontier.at[pos].set(-jnp.inf)
                n_weight=frontier.at[n_pos].get().item()
                if graph.edges[(v,neighbor)]<n_weight:
                  parents[neighbor]=v
                  frontier=frontier.at[n_pos].set(graph.edges[(v,neighbor)])
    return tree



In [35]:
frontier=jnp.array([jnp.inf]*10)
frontier=frontier.at[3].set(0.0)

In [36]:
frontier

Array([inf, inf, inf,  0., inf, inf, inf, inf, inf, inf], dtype=float32)

In [37]:
jnp.argmin(frontier[jnp.where(frontier>0)]).item()

0

In [38]:
frontier=frontier.at[5].set(3.0)

In [21]:
G.nodes

{'a', 'b', 'c', 'd'}

In [23]:
G.adjacency_list()

{'d': {'b', 'c'}, 'b': {'a', 'c', 'd'}, 'c': {'a', 'b', 'd'}, 'a': {'b', 'c'}}

In [39]:
jax_prim(G,'a')

3
1
1
1


[(None, 'a', {'weight': 0.0}), ('a', 'b', {'weight': 4.0})]